<a href="https://colab.research.google.com/github/Heity94/WalkieLookie/blob/main/notebooks/PH_add_zip_borugh_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install requirements { display-mode: "form" }
!pip install osmnx -q
!pip install geopy -q

     |████████████████████████████████| 95 kB 3.5 MB/s 
     |████████████████████████████████| 6.3 MB 56.0 MB/s 
     |████████████████████████████████| 1.0 MB 58.4 MB/s 
     |████████████████████████████████| 11.2 MB 36.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 1.0 MB 60.8 MB/s 
     |████████████████████████████████| 16.7 MB 63.4 MB/s 
     |████████████████████████████████| 944 kB 62.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
#@title Imports
import pickle
import folium
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
ox.config(log_console=True, use_cache=True)
from geopy.geocoders import Nominatim
#from networkx.algorithms import approximation as approx # Greedy salesman algorithms
import sklearn
from ast import literal_eval

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Link to common google drive folder (you have to add a link from the foder to your drive first)
data_path = "/content/drive/MyDrive/BPI Lap WalkieLookie/Data/"
#data_path = "/content/drive/MyDrive/WalkieLookie/BPI Lap WalkieLookie/Data" #Roger
#data_path="/home/valentin_werner/Philipps_Playground/WalkieLookie/data/" #Philipp Zeno

# Create own Routing algorithm

### Load data

Our algorithm needs two datasets:


1.   Dataframe of potenital places (nodes) of interest (e.g. all parks in Berlin)
2.   Street data from Berlin (graph data)



In [5]:
#Load DataFrame with nodes of interest (NOI)
places_noi = pd.read_csv(data_path+"parks_bln_complete_clean.csv", index_col=0)
places_noi["nodes_on_graph"]=places_noi.nodes_on_graph.apply(lambda x: literal_eval(x)) #Change type to list

In [6]:
places_noi

,osmid,name,nodes_on_graph,number_nodes_on_graph
3,4413796.0,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259
4,4440110.0,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11
5,4582243.0,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166
6,4582244.0,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103
7,4592806.0,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497
...,...,...,...,...
2350,14121667.0,NaN,"[340411373, 340411484, 340411264, 340411256, 2...",10
2351,14235419.0,East-Side-Park,"[476571126, 476571133, 476571121, 476571134, 4...",74
2352,14236112.0,Schlesischer Busch,"[29787346, 1934347106, 29787347, 262005337, 16...",231
2353,14282253.0,Wassertorplatz Nord,"[3875882034, 3058137110, 2925266614, 292526661...",34


In [8]:
#load street data from berlin
with open(data_path+'graph_berlin.obj', 'rb') as fp:
  street_graph = pickle.load(fp)

## Add center node to each park

In [9]:
# Function to calculate center node (if possible), otherwise return "No center found"
def center_nodes(street_graph, places_nodes_on_map):
  '''Calculate center node of each place of interest, if no center found return string'''
  try: 
    return nx.center(street_graph.subgraph(places_nodes_on_map))[0] #take first center node (in case there are more than 1)
  except:
    return "No center found"

In [11]:
def get_noi(street_graph, places_noi):
  '''Based on the graph data and the places of interest dataframe, check which nodes are also in the street data and calculate the center node.
  Return the dataframe + center nodes as well as a list of nodes to visit (center node per park)
  '''

  street_nodes = set(street_graph.nodes())

  # Create column whihc only includes nodes which are also within the berlin graph dats (streets)
  places_noi["park_nodes_within_graph"] = places_noi.nodes_on_graph.apply(lambda x: [node for node in x if node in street_graph])

  places_noi = places_noi[places_noi.park_nodes_within_graph.apply(lambda x: len(x)) > 0].copy() #Filter out empty parks

  # Get center node for each park
  places_noi["center_node"] = places_noi.park_nodes_within_graph.apply(lambda x: center_nodes(street_graph, x))

  # filter out parks without center (for now)
  #places_noi = places_noi[places_noi.center_node!="No center found"].copy()

  # Select center park nodes and store as list for route planning
  nodes_to_visit = places_noi.loc[:,"center_node"].to_list()

  return places_noi, nodes_to_visit

In [12]:
places_noi, nodes_to_visit = get_noi(street_graph, places_noi)

In [16]:
places_noi = places_noi.drop(columns="park_nodes_within_graph")

In [19]:
# Store results
places_noi.to_csv(data_path+"parks_bln_complete_clean.csv")

In [20]:
places_noi

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node
3,4413796.0,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402
4,4440110.0,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,No center found
5,4582243.0,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986
6,4582244.0,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,No center found
7,4592806.0,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306
...,...,...,...,...,...
2350,14121667.0,NaN,"[340411373, 340411484, 340411264, 340411256, 2...",10,33792358
2351,14235419.0,East-Side-Park,"[476571126, 476571133, 476571121, 476571134, 4...",74,3517797378
2352,14236112.0,Schlesischer Busch,"[29787346, 1934347106, 29787347, 262005337, 16...",231,No center found
2353,14282253.0,Wassertorplatz Nord,"[3875882034, 3058137110, 2925266614, 292526661...",34,5664799035


## Add zip and borugh information to limit number of parks in route calculation

In [55]:
# Lets pick one random park and try to extract zip and borugh info
x = street_graph.nodes()[1392678402]["x"]
y = street_graph.nodes()[1392678402]["y"]


geolocator = Nominatim(user_agent='my-application')
result = geolocator.reverse((y, x))
zip = result.raw['address']["postcode"]
borough = result.raw['address']["borough"]
print(zip)
print(borough)

10707
Charlottenburg-Wilmersdorf


Seems to work, lets put it in a function

In [53]:
# Add zip code and borough
def get_zipcode_borough(df_col, geolocator, street_graph):
    
    if df_col == "No center found":
      return np.nan

    lat = street_graph.nodes()[df_col]["y"]
    lon = street_graph.nodes()[df_col]["x"]

    location = geolocator.reverse((lat, lon))

    zip = location.raw['address'].get("postcode")
    borough = location.raw['address'].get("borough")

    return zip, borough


In [54]:
geolocator = Nominatim(user_agent='my-application')

In [56]:
# Apply function to all parks
places_noi[["zip", "borough"]] = places_noi.apply(lambda x: get_zipcode_borough(x.center_node, geolocator, street_graph), axis=1, result_type="expand")

In [57]:
places_noi

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough
3,4413796.0,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf
4,4440110.0,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,No center found,NaN,NaN
5,4582243.0,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986,10713,Charlottenburg-Wilmersdorf
6,4582244.0,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,No center found,NaN,NaN
7,4592806.0,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306,10557,Mitte
...,...,...,...,...,...,...,...
2350,14121667.0,NaN,"[340411373, 340411484, 340411264, 340411256, 2...",10,33792358,13503,Reinickendorf
2351,14235419.0,East-Side-Park,"[476571126, 476571133, 476571121, 476571134, 4...",74,3517797378,10243,Friedrichshain-Kreuzberg
2352,14236112.0,Schlesischer Busch,"[29787346, 1934347106, 29787347, 262005337, 16...",231,No center found,NaN,NaN
2353,14282253.0,Wassertorplatz Nord,"[3875882034, 3058137110, 2925266614, 292526661...",34,5664799035,10999,Friedrichshain-Kreuzberg


In [58]:
# Store results
places_noi.to_csv(data_path+"parks_bln_complete_clean.csv")

In [63]:
# For ~ 300 parks we did not find a zip code or borough
places_noi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 3 to 2354
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   osmid                  2007 non-null   float64
 1   name                   541 non-null    object 
 2   nodes_on_graph         2007 non-null   object 
 3   number_nodes_on_graph  2007 non-null   int64  
 4   center_node            2007 non-null   object 
 5   zip                    1739 non-null   object 
 6   borough                1739 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 125.4+ KB


In [61]:
# For these 5 nodes we did not find a zip even though we have a borugh
places_noi[(places_noi.zip.isnull())&(places_noi.center_node!="No center found")]

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough
217,27111705.0,NaN,"[698826020, 698826008, 698826011, 698826014, 6...",22,698826008,None,Charlottenburg-Wilmersdorf
369,42148901.0,NaN,"[298942907, 5451255114, 5451255113, 298942908,...",92,2055189218,None,Pankow
689,106653679.0,NaN,"[349010344, 349010340, 9792123874, 3591670851,...",53,8991933104,None,Charlottenburg-Wilmersdorf
961,217673569.0,Alice-Herz-Platz,"[4127743704, 4127743706, 4127743709, 412774371...",10,4127743704,None,Marzahn-Hellersdorf
2316,8726382.0,Bürgerpark Marzahn,"[8112781709, 8430661292, 8430661302, 363598728...",398,1611104577,None,Marzahn-Hellersdorf


For these 300 parks lets just take a random node from the park and call the function agin with this node to get the missing data

In [87]:
tst = places_noi.loc[places_noi.zip.isna()].apply(lambda x: get_zipcode_borough(x.nodes_on_graph[0], geolocator, street_graph), axis=1, result_type="expand")

In [96]:
tst.columns = ["zip_0", "borough_0"]

In [110]:
places_noi_mrg = places_noi.merge(tst, how="left", left_index=True, right_index=True)

In [111]:
places_noi_mrg.loc[places_noi_mrg.zip.isna(), "zip"] = places_noi_mrg.loc[places_noi_mrg.zip.isna(), "zip_0"]
places_noi_mrg.loc[places_noi_mrg.borough.isna(), "borough"] = places_noi_mrg.loc[places_noi_mrg.borough.isna(), "borough_0"]

In [115]:
places_noi_mrg = places_noi_mrg.drop(columns=["zip_0", "borough_0"])

In [120]:
places_noi_mrg[(places_noi_mrg.zip.isna())|(places_noi_mrg.borough.isna())]

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough
217,2.711170e+07,NaN,"[698826020, 698826008, 698826011, 698826014, 6...",22,698826008,None,Charlottenburg-Wilmersdorf
398,4.393450e+07,NaN,"[353674872, 250931430, 701388077, 4757710701, ...",340,No center found,None,Marzahn-Hellersdorf
961,2.176736e+08,Alice-Herz-Platz,"[4127743704, 4127743706, 4127743709, 412774371...",10,4127743704,None,Marzahn-Hellersdorf
1069,2.812833e+08,NaN,"[8892396970, 4529158852, 281399739, 4529158843...",9,281399740,15566,NaN
1083,2.930153e+08,NaN,"[7858501172, 7858501171]",2,7858501171,15566,NaN
1433,5.429413e+08,NaN,"[80384236, 303346744, 347393991, 303346745, 30...",57,1872319024,16356,NaN
1713,6.616594e+08,NaN,"[901472006, 8954167376, 901472013, 901471999, ...",29,3076108552,14532,NaN
2155,1.043339e+09,NaN,"[7788171433, 7788171441, 7788171435, 778824625...",33,9602441622,14612,NaN


In [135]:
places_noi_mrg.loc[places_noi_mrg.borough.isin(["Charlottenburg-Wilmersdorf", "Marzahn-Hellersdorf"])].groupby(["borough", "zip"])[["osmid"]].count().sort_values("osmid").groupby(level=0).tail(1)

,,osmid
borough,zip,
Charlottenburg-Wilmersdorf,14193,24
Marzahn-Hellersdorf,12683,44


In [140]:
# Set them manually
places_noi_mrg.loc[217, "zip"] = 14193

In [142]:
places_noi_mrg.loc[[398, 961], "zip"] = 12683

In [145]:
# Store results
places_noi_mrg.to_csv(data_path+"parks_bln_complete_clean.csv")